<a href="https://colab.research.google.com/github/IgorBaratta/simple_fem/blob/master/demo/Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install git+https://github.com/IgorBaratta/simple_fem.git

Problem definition:

$$- \nabla^2 u = f \quad \text{in} \quad \Omega $$
$$   u = 0 \quad \text{in} \quad \partial \Omega $$

---


Where $\Omega:= (0, 1) \times (0, 1)$ and $f = 4 (-y^2 + y ) sin (\pi x)$


In [0]:
import numpy
from simple_fem import *
from scipy.sparse.linalg import spsolve
from simple_fem.assemble import assemble_matrix, assemble_vector, apply_bc

# define computational domain - Unit Square
mesh = Mesh(25, 20)
plot(mesh)

The problem can be rewritten using the finite element framework. 

Find $u_h \in Q$ such that:

$$\int_\Omega \nabla u_h \cdot \nabla v = \int fv $$

We first define the discrete function space:


In [0]:
# Define element and function space
element = Q1Element()
Q = FunctionSpace(mesh, element)

print("Number of dofs per element: ", element.num_dofs)

The source term $f = 4 (-y^2 + y ) sin (\pi x)$ can be represented by a lambda function. To assemble the linear form $$b_i = \int_{\Omega} f \hat{\phi}_i$$ we then call `assemble_vector`:


In [0]:
f = lambda x : 4*(-x[1]**2 + x[1])*numpy.sin(numpy.pi*x[0])
b = assemble_vector(Q, f)

Likewise we can assemble the bilinear form $$A_{ij} = \int_\Omega \nabla \phi_j \cdot \nabla \phi_i \, dx$$

by calling the `assemble_matrix` function with the `matrix_type` set to "stiffness":


In [0]:
A = assemble_matrix(Q, matrix_type="stiffness")

In [0]:
# Finally we can apply Dirichlet boundary conditions
# and call a sparse linear solver from scipy, for example spsolve:

dofs = Q.locate_boundary_dofs()
apply_bc(A, b, dofs, value=0)
x = spsolve(A, b)

plot(mesh, x)